## 8. 完整策略

In [1]:
import rqalpha
import os
from rqalpha.api import *
import talib
import pandas as pd


def init(context):
    context.s1 = "000001.XSHE"
    context.SHORTPERIOD = 10
    context.LONGPERIOD = 30

def handle_bar(context, bar_dict):
    prices = history_bars(context.s1, context.LONGPERIOD+1, '1d', 'close')
    short_avg = talib.SMA(prices, context.SHORTPERIOD)
    long_avg = talib.SMA(prices, context.LONGPERIOD)

    # 计算现在portfolio中股票的仓位
    cur_position = context.portfolio.positions[context.s1].quantity
    avg_price = context.portfolio.positions[context.s1].avg_price
    capital = cur_position*avg_price
    # 计算现在portfolio中的现金可以购买多少股票
    shares = context.portfolio.cash/bar_dict[context.s1].close
    #图形显示当前占用资金
    plot('capital', capital)

    # 如果短均线从上往下跌破长均线，而上一个bar的短线平均值高于长线平均值
    if short_avg[-1] - long_avg[-1] < 0 and short_avg[-2] - long_avg[-2] > 0 and cur_position > 0:
        # 进行清仓
        order_target_value(context.s1, 0)

    # 如果短均线从下往上突破长均线，为入场信号
    if short_avg[-1] - long_avg[-1] > 0 and short_avg[-2] - long_avg[-2] < 0:
        # 满仓入股
        order_shares(context.s1, shares)

config = {
  "base": {
    "start_date": "2015-06-01",
    "end_date": "2017-12-30",
    "accounts": {'stock':1000000},
    "benchmark": "000001.XSHE",
#     "strategy_file_path": os.path.abspath(__file__)
  },
  "extra": {
    "log_level": "error",
  },
  "mod": {
    "sys_analyser": {
      "enabled": True,
      "plot": True
    }
  }
}

# 您可以指定您要传递的参数
rqalpha.run_func(init=init, handle_bar=handle_bar, config=config)

<Figure size 1800x1000 with 4 Axes>

{'sys_analyser': {'summary': {'strategy_name': 'strategy',
   'start_date': '2015-06-01',
   'end_date': '2017-12-29',
   'strategy_file': 'strategy.py',
   'run_type': 'BACKTEST',
   'benchmark': '000001.XSHE',
   'STOCK': 1000000.0,
   'alpha': -0.06,
   'beta': 0.041,
   'sharpe': -0.98,
   'information_ratio': -0.302,
   'downside_risk': 0.207,
   'tracking_error': 0.291,
   'sortino': -0.284,
   'volatility': 0.06,
   'max_drawdown': 0.154,
   'total_value': 910939.634,
   'cash': 910939.634,
   'total_returns': -0.089,
   'annualized_returns': -0.035,
   'unit_net_value': 0.911,
   'units': 1000000.0,
   'benchmark_total_returns': 0.025,
   'benchmark_annualized_returns': 0.01},
  'trades':                      commission         exec_id  last_price  last_quantity  \
  datetime                                                                     
  2015-06-04 15:00:00     798.856  15337470450000       16.37          61000   
  2015-06-23 15:00:00     729.560  15337470450001       